In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

In [3]:
dataset_path = "/Users/yehormishchyriak/Desktop/qac239_final_project/clean_data.csv"

In [4]:
df = pd.read_csv(dataset_path)

In [5]:
# image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 for pretrained CNNs
    transforms.ToTensor(),         # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for pretrained models
])

# Load and preprocess images
def preprocess_image(image_index):
    image = Image.open(f"/Users/yehormishchyriak/Desktop/qac239_final_project/album_covers/{image_index}.jpg").convert('RGB')  # Ensure image is RGB
    return transform(image)

# Apply preprocessing
df['image_tensor'] = df['album_id'].apply(preprocess_image)

In [6]:
df

,album_id,genre,image_tensor
0,0,0,"[[[tensor(0.9646), tensor(0.9474), tensor(0.96..."
1,1,0,"[[[tensor(-1.2274), tensor(-1.1589), tensor(-1..."
2,2,1,"[[[tensor(2.2489), tensor(2.2489), tensor(2.24..."
3,4,0,"[[[tensor(-1.3644), tensor(-1.4500), tensor(-1..."
4,6,4,"[[[tensor(0.5022), tensor(0.5193), tensor(0.50..."
...,...,...,...
1603,2155,1,"[[[tensor(-2.1008), tensor(-2.1008), tensor(-2..."
1604,2157,2,"[[[tensor(-0.1143), tensor(-0.1143), tensor(-0..."
1605,2159,3,"[[[tensor(-0.2856), tensor(-0.2856), tensor(-0..."
1606,2160,4,"[[[tensor(2.1290), tensor(2.1633), tensor(0.89..."


In [7]:
df.to_csv("album_covers_dataset.csv", index=False)

### Intended split: Training - 80%, Validation - 10%, Testing - 10%.

In [8]:
# split into 80% training and 20% temp (validation + testing)
train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)

# split temp data into 50% validation and 50% testing
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print("Training Set Size:", len(train_data))
print("Validation Set Size:", len(validation_data))
print("Testing Set Size:", len(test_data))

Training Set Size: 1286
Validation Set Size: 161
Testing Set Size: 161


## DATASET:

In [9]:
class AlbumDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get image and label
        image_path = (f"/Users/yehormishchyriak/Desktop/qac239_final_project/album_covers/{self.dataframe.iloc[idx]['album_id']}.jpg")
        label = self.dataframe.iloc[idx]['genre']
        
        # Load and transform image
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        return image, label